<a href="https://colab.research.google.com/github/c-bujari/CAP4630/blob/master/HW_5/HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 5
###CAP 4630 Artificial Intelligience
####Clyde Bujari

## 1. General Concepts

**Artificial Intelligience** is a concept that has had many definitions by many different people over the years - but the most accurate one is John McCarthy's definition of it as "the science and engineering of making intelligient machines". 

In other words, the study of AI is the study of designing computer systems which can perform tasks such as visual perception, speech recognition, decision making, etc.; tasks which are virtually impossible to emulate with traditional computer programming, but that human intelligience is naturally good at.

**Machine Learning** is a subset of AI. Arthur Samuel defined it as a "field of study that gives computers the ability to learn without being explicitely programmed". The goal of ML is to create programs, more specifically, which adjust their output in response to a given set of input data, much like humans learn from birth. These programs are more flexible than those created by human programmers, able to change themselves based on data and thus create much more feasible solutions to problems AI seeks to solve.

**Deep Learning** is a subset of ML that specifically uses algorithms broken into  multiple layers (hence the term "deep"). By breaking up an algorithm into these layers, each layer can handle a different part of the process - for example, in image processing a lower layer may handle edge identification, and a higher layer may identify which of these edges correspond to a face or a digit.

## 2. Basic Concepts

**Linear regression** is a statistical tool used to model the relationship between a dependent variable and one or more independent variables.

**Logistic regression**, is a statistical tool that is used to model the probability of a certain class/event being valid - pass/fail, win/lose, etc. This can also be extended to more than just two binary classes, such as to determine what animal an image contains (assigning each possible class a probability between 0 and 1).

The **gradient** of a multivariate function is a vector field for which the components of a vector at a given point are the partial derivatives of function f. It is the multivariate equivalent to a "derivative"; the gradient vector's direction = the direction of fastest increase of f, and the gradient vector's magnitude = the rate of increase in that direction.

**Gradient descent** (also known as *steepest descent*) is an iterative algorithm for finding a local minimum of any differentiable function. In each iteration, it updates the value of this minimum by decreasing by a step size, where the step = the gradient at the current point * a scalar learning rate.

## 3. Building a model

## 4. Compiling a model

## 5. Training a model

## 6. Finetuning a pretrained model